In [ ]:
# FAQ Chatbot using NLP and Cosine Similarity

# Install required libraries
!pip install nltk scikit-learn ipywidgets -q

print("Packages installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.4 MB/s eta 0:00:00
Packages installed successfully!


In [ ]:
# Import all necessary libraries

import nltk
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

print("Libraries imported successfully!")

Libraries imported successfully!


In [ ]:
# Download required NLTK datasets for text processing

print("Downloading NLTK data...")

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

print("\n NLTK data downloaded successfully!")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...



 NLTK data downloaded successfully!


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# Add your FAQs here - You can modify this list with your own questions and answers

faqs = [
    {
        "question": "What are your business hours?",
        "answer": "We are open Monday through Friday from 9 AM to 6 PM, and Saturday from 10 AM to 4 PM. We are closed on Sundays."
    },
    {
        "question": "How can I track my order?",
        "answer": "You can track your order by logging into your account and visiting the 'My Orders' section. You'll receive a tracking number via email once your order ships."
    },
    {
        "question": "What is your return policy?",
        "answer": "We accept returns within 30 days of purchase. Items must be unused and in original packaging. Refunds are processed within 5-7 business days."
    },
    {
        "question": "Do you offer international shipping?",
        "answer": "Yes, we ship to over 50 countries worldwide. Shipping costs and delivery times vary by location. International orders typically arrive within 7-14 business days."
    },
    {
        "question": "How do I reset my password?",
        "answer": "Click on 'Forgot Password' on the login page. Enter your email address, and we'll send you a link to reset your password."
    },
    {
        "question": "What payment methods do you accept?",
        "answer": "We accept all major credit cards (Visa, MasterCard, American Express), PayPal, and Apple Pay."
    },
    {
        "question": "Can I cancel my order?",
        "answer": "You can cancel your order within 24 hours of placing it. After that, the order may have already been processed for shipping. Contact customer support for assistance."
    },
    {
        "question": "Do you have a mobile app?",
        "answer": "Yes! Our mobile app is available for both iOS and Android. You can download it from the App Store or Google Play Store."
    },
    {
        "question": "How do I contact customer support?",
        "answer": "You can reach our customer support team via email at support@example.com or call us at 1-800-123-4567 during business hours."
    },
    {
        "question": "Are there any discounts for bulk orders?",
        "answer": "Yes, we offer discounts for bulk orders. Please contact our sales team at sales@example.com for a custom quote."
    }
]

print(f" Loaded {len(faqs)} FAQs")

 Loaded 10 FAQs


In [ ]:
# This class handles text cleaning and preprocessing

class TextPreprocessor:
    """
    Preprocesses text for better matching:
    - Converts to lowercase
    - Removes special characters
    - Tokenizes text
    - Removes stop words (common words like 'the', 'is', etc.)
    - Lemmatizes words (converts to base form)
    """

    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))

    def preprocess(self, text):
        """Clean and preprocess text"""
        # Convert to lowercase
        text = text.lower()

        # Remove special characters and digits
        text = re.sub(r'[^a-zA-Z\s]', '', text)

        # Tokenize (split into words)
        tokens = word_tokenize(text)

        # Remove stopwords and lemmatize
        tokens = [
            self.lemmatizer.lemmatize(word)
            for word in tokens
            if word not in self.stop_words and len(word) > 2
        ]

        return ' '.join(tokens)

# Test the preprocessor
preprocessor = TextPreprocessor()
sample_text = "How can I track my order? What's the status?"
processed = preprocessor.preprocess(sample_text)

print(" TextPreprocessor class created")
print(f"\nExample:")
print(f"Original: {sample_text}")
print(f"Processed: {processed}")

 TextPreprocessor class created

Example:
Original: How can I track my order? What's the status?
Processed: track order whats status


In [ ]:
# This class finds the best matching FAQ for a user's question

class FAQMatcher:
    """
    Matches user questions to FAQs using:
    - TF-IDF Vectorization (converts text to numerical vectors)
    - Cosine Similarity (measures similarity between vectors)
    """

    def __init__(self, faqs):
        self.faqs = faqs
        self.preprocessor = TextPreprocessor()
        self.vectorizer = TfidfVectorizer()

        # Preprocess all FAQ questions
        self.faq_questions = [faq['question'] for faq in faqs]
        self.preprocessed_questions = [
            self.preprocessor.preprocess(q) for q in self.faq_questions
        ]

        # Create TF-IDF vectors for FAQ questions
        self.faq_vectors = self.vectorizer.fit_transform(self.preprocessed_questions)

    def find_best_match(self, user_question, threshold=0.3):
        """
        Find the best matching FAQ for user question

        Args:
            user_question: The question asked by user
            threshold: Minimum similarity score (0-1) to consider a match

        Returns:
            Dictionary with matched question, answer, and confidence score
            or None if no good match found
        """
        # Preprocess user question
        processed_question = self.preprocessor.preprocess(user_question)

        # Convert to TF-IDF vector
        user_vector = self.vectorizer.transform([processed_question])

        # Calculate cosine similarity with all FAQ questions
        similarities = cosine_similarity(user_vector, self.faq_vectors)[0]

        # Find best match
        best_match_idx = np.argmax(similarities)
        best_similarity = similarities[best_match_idx]

        # Return result if above threshold
        if best_similarity >= threshold:
            return {
                'question': self.faq_questions[best_match_idx],
                'answer': self.faqs[best_match_idx]['answer'],
                'confidence': round(best_similarity * 100, 2)
            }
        else:
            return None

# Test the matcher
matcher = FAQMatcher(faqs)
test_question = "Where can I find my order status?"
result = matcher.find_best_match(test_question)

print(" FAQMatcher class created")
print(f"\nExample:")
print(f"User Question: {test_question}")
if result:
    print(f"Matched FAQ: {result['question']}")
    print(f"Confidence: {result['confidence']}%")

 FAQMatcher class created

Example:
User Question: Where can I find my order status?
Matched FAQ: How can I track my order?
Confidence: 59.68%


In [ ]:
# This class creates the interactive chat interface

class FAQChatbot:
    """
    Creates an interactive chatbot UI with:
    - Text input field
    - Send button
    - Clear chat button
    - Chat history display
    """

    def __init__(self, faqs):
        self.matcher = FAQMatcher(faqs)
        self.chat_history = []

        # Create UI widgets
        self.output = widgets.Output()
        self.user_input = widgets.Text(
            placeholder='Type your question here...',
            description='You:',
            layout=widgets.Layout(width='70%')
        )
        self.send_button = widgets.Button(
            description='Send',
            button_style='primary',
            layout=widgets.Layout(width='15%')
        )
        self.clear_button = widgets.Button(
            description='Clear Chat',
            button_style='warning',
            layout=widgets.Layout(width='15%')
        )

        # Event handlers
        self.send_button.on_click(self.on_send_click)
        self.clear_button.on_click(self.on_clear_click)
        self.user_input.on_submit(self.on_send_click)

    def display_message(self, sender, message, confidence=None):
        """Display a message in the chat with styling"""
        with self.output:
            if sender == "You":
                style = "background-color: #e3f2fd; padding: 10px; margin: 5px; border-radius: 10px;"
                display(HTML(f'<div style="{style}"><strong>{sender}:</strong> {message}</div>'))
            else:
                style = "background-color: #f1f8e9; padding: 10px; margin: 5px; border-radius: 10px;"
                conf_text = f" <em>(Confidence: {confidence}%)</em>" if confidence else ""
                display(HTML(f'<div style="{style}"><strong>{sender}:</strong> {message}{conf_text}</div>'))

    def on_send_click(self, b):
        """Handle send button click or Enter key press"""
        user_question = self.user_input.value.strip()

        if not user_question:
            return

        # Display user message
        self.display_message("You", user_question)
        self.chat_history.append(("user", user_question))

        # Find matching FAQ
        match = self.matcher.find_best_match(user_question)

        if match:
            response = f"<strong>Q:</strong> {match['question']}<br><br><strong>A:</strong> {match['answer']}"
            self.display_message("Bot", response, match['confidence'])
            self.chat_history.append(("bot", match['answer']))
        else:
            response = "I'm sorry, I couldn't find a relevant answer to your question. Please try rephrasing or contact our support team for assistance."
            self.display_message("Bot", response)
            self.chat_history.append(("bot", response))

        # Clear input field
        self.user_input.value = ''

    def on_clear_click(self, b):
        """Clear chat history"""
        self.chat_history = []
        with self.output:
            clear_output()
            display(HTML('<h3 style="color: #1976d2;">FAQ Chatbot</h3>'))
            display(HTML('<p>Hello! I\'m here to help answer your questions. Type your question below.</p>'))

    def run(self):
        """Display the chatbot UI"""
        # Display header
        with self.output:
            display(HTML('<h3 style="color: #1976d2;">FAQ Chatbot</h3>'))
            display(HTML('<p>Hello! I\'m here to help answer your questions. Type your question below.</p>'))

        # Create layout
        input_box = widgets.HBox([self.user_input, self.send_button, self.clear_button])
        ui = widgets.VBox([self.output, input_box])

        display(ui)

print("FAQChatbot class created")


FAQChatbot class created


In [ ]:
# Execute this cell to start the chatbot

print("🤖 Initializing FAQ Chatbot...")


# Create and run chatbot
chatbot = FAQChatbot(faqs)
chatbot.run()


print(" Chatbot is ready! Start asking questions above.")

print("\nTips:")
print("- Type your question and press Enter or click Send")
print("- The bot will find the most similar FAQ")
print("- Confidence score shows how good the match is")
print("- Click 'Clear Chat' to reset the conversation")

🤖 Initializing FAQ Chatbot...


 Chatbot is ready! Start asking questions above.

Tips:
- Type your question and press Enter or click Send
- The bot will find the most similar FAQ
- Confidence score shows how good the match is
- Click 'Clear Chat' to reset the conversation


In [ ]:
faqs = {
    "What is the admission deadline?": "The admission deadline is 30th September 2025.",
    "How can I apply for admission?": "You can apply through our online admission portal.",
    "What documents are required for admission?": "You need your transcript, CNIC, and passport-sized photos.",
    "Is there any scholarship available?": "Yes, we offer merit-based scholarships.",
    "How can I contact the admission office?": "You can email us at admission@university.edu."
}


In [ ]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load SpaCy English model
nlp = spacy.load("en_core_web_sm")

# Extract questions
questions = list(faqs.keys())

# Vectorize questions using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(questions)


In [ ]:
def chatbot_response(user_query):
    # Transform query
    user_vec = vectorizer.transform([user_query])

    # Compute cosine similarity
    similarity = cosine_similarity(user_vec, X).flatten()

    # Find best match
    idx = similarity.argmax()
    best_match_score = similarity[idx]

    # Confidence threshold (to avoid wrong matches)
    if best_match_score > 0.3:
        return faqs[questions[idx]]
    else:
        return "Sorry, I don’t have an answer for that. Please contact support."


In [ ]:
print("Chatbot ready! Type 'exit' to quit.\n")
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break
    print("Bot:", chatbot_response(user_input))


Chatbot ready! Type 'exit' to quit.

